

```
# Dit is opgemaakt als code
```

# DCGAN to generate grayscale images


## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import cv2
import random as rd
import PIL

# CHANGE

_____________________________________________________________________

In [ ]:
# Image size (height x width)
ih = 128
iw = 128

# Grayscale or RGB
ch = 'rgb'

# Layer adapt
ksize = 4 # Kernel size : was '4' for 64x64 image
ssize = 2 # Stride size : was '2' for 64x64 image

# Batch size
batch_size = 32

# TPU
print('Are you going to use TPU?')
use_tpu = input("y/n")

_____________________________________________________________________

**TPU Setup (Optional & Automatic)**

In [ ]:
if(use_tpu == 'y' or use_tpu == 'Y'):
    # Detect and init. TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

    # Initialize distributed TPU strategy
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
print("Tensorflow version " + tf.__version__)

In [ ]:
try: 
    os.mkdir("/kaggle/working/checkpoints")
    print("Path for checkpoints has been created!")
except:
    print("Path already exists")

In [ ]:
try:
    os.mkdir("/kaggle/working/generated_images")
    print("Path for generated images has been created!")
except:
    print("Path already exists")

## Prepare CelebA data

We'll use face images from the CelebA dataset, resized to 64x64.

In [ ]:
# Overwrite if TPU is used
if use_tpu == 'y' or use_tpu == 'Y':
    batch_size = 16 * tpu_strategy.num_replicas_in_sync

In [ ]:
if use_tpu == 'y' or use_tpu == 'Y':
    # Step 1: Get the credentail from the Cloud SDK
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    
    # Step 2: Set the credentials
    user_secrets.set_tensorflow_credential(user_credential)
    
    # Step 3: Use a familiar call to get the GCS path of the dataset
    !gcloud config set project 'solid-topic-344315'
    
    GCS_DS_PATH = 'gs://thesis_data_max/classified_data'

    
else: path_root = '/kaggle/input/thesis-data'

Create a dataset from our folder, and rescale the images to the [0-1] range:

In [ ]:
im_si = (ih, iw)

if(ch == 'rgb'):
    chnum = 3
elif(ch == 'grayscale'):
    chnum = 1


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    path_root, 
    label_mode = None,
    color_mode = ch,
    image_size = im_si,
    interpolation='bicubic',
    batch_size = batch_size
)

dataset = dataset.map(lambda x : x / 255.0)

Let's display a sample image:

## Create the discriminator

It maps a 64x64 image to a binary classification score.

In [ ]:
def create_discriminator():
  return keras.Sequential(
      [
        
          layers.Conv2D(ih, kernel_size=ksize, strides=ssize, padding="same",
                        input_shape=(ih, iw, chnum)),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(2*ih, kernel_size=ksize, strides=ssize, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(2*ih, kernel_size=ksize, strides=ssize, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Flatten(),
          layers.Dropout(0.2),
          layers.Dense(1, activation="sigmoid"),
      ],
      name="discriminator",
  )


## Create the generator

It mirrors the discriminator, replacing `Conv2D` layers with `Conv2DTranspose` layers.

In [ ]:
latent_dim = 2*ih
sih = ih//8
siw = iw//8

def create_generator():
  return keras.Sequential(
      [
          keras.layers.InputLayer(input_shape=(latent_dim)),
          
          layers.Dense(sih * siw * latent_dim),
          layers.Reshape((sih, siw, latent_dim)),
          layers.Conv2DTranspose(latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(2*latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(4*latent_dim, kernel_size=ksize, strides=ssize, padding="same"),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(chnum, kernel_size=5, padding="same", activation="sigmoid"),
      ],
      name="generator",
  )


In [ ]:
if use_tpu == 'y' or use_tpu == 'Y':
    with tpu_strategy.scope():
        generator = create_generator()
        discriminator = create_discriminator()
else:  
    generator = create_generator()
    discriminator = create_discriminator()

## Override `train_step`

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }
  


**Other GAN Script**

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [ ]:
def discriminator_loss(real_output, fake_output, d_loss):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    d_loss.append(total_loss)
    return total_loss

In [ ]:
def generator_loss(fake_output, g_loss):
    fake_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    g_loss.append(fake_loss)
    return fake_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
discriminator.summary()

In [ ]:
generator.summary()

In [ ]:
checkpoint_dir = '/kaggle/working/checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=create_generator(),
                                 discriminator=create_discriminator())

## Create a callback that periodically saves generated images

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=latent_dim):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        start = time.time()
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("/kaggle/working/generated_images/generated_img_%03d_%d.png" % (epoch, i))
    
        # Save the model every 15 epochs (WAS 15)
        if (epoch + 1) % 15 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)



## Train the end-to-end model

In [ ]:
epochs = 90  # In practice, use ~100 epochs

if use_tpu == 'y' or use_tpu == 'Y':
    with tpu_strategy.scope():
        gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

        gan.compile(
        d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
        loss_fn=keras.losses.BinaryCrossentropy(),
        )
        
else: 
    gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)

    gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Was 0.0001
    loss_fn=keras.losses.BinaryCrossentropy(),
    )

#steps_per_epoch = 2500 // batch_size
gan.fit(
  dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)

In [ ]:
for x in dataset:
  i = rd.randint(0,2512)
  plt.imshow((x.numpy() * 255).astype("int32")[0])
  break

Some of the last generated images around epoch 30
(results keep improving after that):


# Checkpoint Restore

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))